In [91]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

library("car")
require("lattice")
require("ez")

library("cowplot")
library("ggbeeswarm")

library("nortest")

library("effectsize")

theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))


options(width=240)
library(tidyverse)


In [92]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")

In [93]:
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}

In [94]:
AOV4 <- function(DATA,INFO){

    INFO <- paste("`",INFO,"`",sep="")
    DATA_SUMMARY <- DATA %>%
        group_by(cond0) %>%
        summarise(
            N = n(),
            valX_M = mean(valX),
            valX_SD = sd(valX),
            valX_SE = valX_SD / sqrt(N),
            .groups = "drop"
        )    
    DATA_aov <- aov_4(
        formula  <- FORMULA,
        data     <- DATA,
    )    
    DATA_aov_APA <- apa_print(DATA_aov)
    DATA_aov_EMM <- emmeans(DATA_aov, specs<-c("cond0"))
    DATA_aov_EMM_APA <- apa_print(DATA_aov_EMM)
    DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM)
    DATA_aov_EMM_PAIRS_APA <- apa_print(DATA_aov_EMM_PAIRS)    

    return(list("info"      = INFO,
                "summary"   = DATA_SUMMARY,
                "aov"       = DATA_aov,
                "aov_APA"   = DATA_aov_APA,
                "emm"       = DATA_aov_EMM,
                "emm_APA"   = DATA_aov_EMM_APA,
                "pairs"     = DATA_aov_EMM_PAIRS,
                "pairs_APA" = DATA_aov_EMM_PAIRS_APA
               )
          )
}

# Peaks 0

In [95]:
ifName="./data/JOB_004/exports/df3_peaks3.csv"

df0 <- read.csv(file = ifName)
df0 <- subset(df0, CHAN_BUND=="" )
df0 <- subset(df0, quest0=="word_set" )

In [6]:
df1 <- data.frame(df0)

In [7]:
df1 <- subset(df1, cond0 %in% c("noun_F","noun_M","verb_X") )
# df1 <- subset(df1, cond0 %in% c("noun_X","verb_X") )

In [8]:
'%!in%' <- function(x,y)!('%in%'(x,y))

# df1 <- subset(df1, SUB %!in% c("01mkxf","02ppxm","09msxf","15agxf","17makf","24jkxf","26mwxf","31ltxf","33kkxm","37lpxm") )

In [9]:
df1 <- droplevels(df1)
# df1 <- subset(df1, select = -c(evoked0,SES,TASK,RUN,chanX,quest0,CHAN_BUND) )
df1 <- subset(df1, select = c(cond0,chan0,tmin0,tmax0,mode0,valX,SUB) )


In [10]:
str(df1)

'data.frame':	9072 obs. of  7 variables:
 $ cond0: Factor w/ 3 levels "noun_F","noun_M",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ chan0: Factor w/ 6 levels "LC","LF","LP",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ tmin0: num  0.1 0.1 0.19 0.19 0.35 0.35 0.41 0.41 0.6 0.6 ...
 $ tmax0: num  0.19 0.19 0.29 0.29 0.45 0.45 0.58 0.58 0.9 0.9 ...
 $ mode0: Factor w/ 2 levels "neg","pos": 2 1 2 1 2 1 2 1 2 1 ...
 $ valX : num  0.0756 -2.2746 -0.0233 -1.0445 -1.4546 ...
 $ SUB  : Factor w/ 42 levels "01mkxf","02ppxm",..: 1 1 1 1 1 1 1 1 1 1 ...


In [11]:
summary(df1["cond0"])

    cond0     
 noun_F:3024  
 noun_M:3024  
 verb_X:3024  

In [12]:
c(unique(df1["chan0"]))

$chan0
[1] LF RF LC RC LP RP
Levels: LC LF LP RC RF RP

In [13]:
c(unique(df1["tmin0"]))

$tmin0
[1] 0.10 0.19 0.35 0.41 0.60 0.11

https://www.rdocumentation.org/packages/afex/versions/0.27-2/topics/aov_car

In [14]:
FORMULA  <- valX~(cond0|SUB)
TEST     <- adjusted("free")

In [15]:
CONDS <- c()
mode0 <- as.matrix(unique(df1["mode0"]))
chan0 <- as.matrix(unique(df1["chan0"]))
tmin0 <- as.matrix(unique(df1["tmin0"]))
for(tmin1 in tmin0 ){
    for(chan1 in chan0 ){
        for(mode1 in mode0 ){
            CONDS <- c(CONDS,str_c("tmin0==",tmin1," & chan0==\"",chan1,"\" & mode0==\"",mode1,"\""))
        }
    }
}

In [16]:
length(CONDS)

[1] 72

In [17]:
IDX=0
CONDS[15]

[1] "tmin0==0.19 & chan0==\"RF\" & mode0==\"pos\""

In [18]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
01mkxf, 05jwxf, 07zrxf, 09msxf, 14ktxf, 16olxm, 20jpxf, 23zdxf, 30mszf, 38slxm
Removing those cases from the analysis.”


# Condition 1 of 72

`tmin0==0.1 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.95,1.76,36.95,0.65,.071,.022


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,28,0.5275721,1.686444,0.3187080
noun_M,27,0.1631923,1.636045,0.3148569
verb_X,27,0.1335126,1.226710,0.2360805


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.70,"$[0.03$, $1.36]$",2.15,.040
noun_M,noun_M,0.30,"$[-0.36$, $0.97]$",0.93,.358
verb_X,verb_X,0.16,"$[-0.50$, $0.83]$",0.50,.619


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.39,"$[-0.16$, $0.94]$",1.73,.207
noun_F_verb_X,noun_F - verb_X,0.53,"$[-0.02$, $1.09]$",2.34,.061
noun_M_verb_X,noun_M - verb_X,0.14,"$[-0.41$, $0.69]$",0.62,.812


In [19]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
15agxf, 24jkxf, 39mrxm
Removing those cases from the analysis.”


# Condition 2 of 72

`tmin0==0.1 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.06,1.88,71.50,0.27,.350,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,-2.894785,1.309856,0.2071064
noun_M,41,-2.666083,1.699801,0.2654642
verb_X,42,-2.669956,1.454448,0.2244262


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.93,"$[-3.40$, $-2.46]$",-12.50,< .001
noun_M,noun_M,-2.77,"$[-3.24$, $-2.30]$",-11.81,< .001
verb_X,verb_X,-2.82,"$[-3.29$, $-2.35]$",-12.04,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.16,"$[-0.43$, $0.11]$",-1.42,.334
noun_F_verb_X,noun_F - verb_X,-0.11,"$[-0.38$, $0.16]$",-0.96,.602
noun_M_verb_X,noun_M - verb_X,0.05,"$[-0.22$, $0.32]$",0.46,.890


In [20]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
01mkxf, 09msxf, 11kbxm, 16olxm, 17makf, 20jpxf, 27zgxf, 29jjxf, 30mszf, 32mjxf, 34gsxm, 37lpxm, 42aszm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 3 of 72

`tmin0==0.1 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.99,1.99,47.71,0.40,.148,.010


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,34,-0.09585035,1.688673,0.2896050
noun_M,35,-0.41178819,1.626501,0.2749288
verb_X,29,0.33669843,1.590460,0.2953411


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.47,"$[-0.13$, $1.06]$",1.60,.121
noun_M,noun_M,0.15,"$[-0.44$, $0.74]$",0.51,.611
verb_X,verb_X,0.45,"$[-0.14$, $1.05]$",1.54,.133


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.32,"$[-0.12$, $0.75]$",1.77,.191
noun_F_verb_X,noun_F - verb_X,0.01,"$[-0.42$, $0.45]$",0.08,.996
noun_M_verb_X,noun_M - verb_X,-0.30,"$[-0.73$, $0.13]$",-1.69,.221


In [21]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
39mrxm
Removing those cases from the analysis.”


# Condition 4 of 72

`tmin0==0.1 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.15,1.74,69.46,0.36,.828,.000


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.844659,1.502761,0.2318811
noun_M,42,-2.856934,1.447292,0.2233220
verb_X,41,-2.849458,1.580045,0.2467616


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.92,"$[-3.39$, $-2.45]$",-12.51,< .001
noun_M,noun_M,-2.89,"$[-3.36$, $-2.42]$",-12.38,< .001
verb_X,verb_X,-2.85,"$[-3.32$, $-2.38]$",-12.22,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.03,"$[-0.32$, $0.26]$",-0.25,.966
noun_F_verb_X,noun_F - verb_X,-0.07,"$[-0.36$, $0.23]$",-0.55,.845
noun_M_verb_X,noun_M - verb_X,-0.04,"$[-0.33$, $0.26]$",-0.30,.950


In [22]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf, 10mczf, 18skxm, 20jpxf, 22foxm, 32mjxf, 40asxm, 42aszm
Removing those cases from the analysis.”


# Condition 5 of 72

`tmin0==0.1 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.81,1.71,54.66,0.25,.177,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,39,0.8265972,1.1068816,0.1772429
noun_M,38,0.7272878,0.9466989,0.1535748
verb_X,37,0.8233408,1.1723081,0.1927263


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.92,"$[0.56$, $1.28]$",5.22,< .001
noun_M,noun_M,0.73,"$[0.38$, $1.09]$",4.16,< .001
verb_X,verb_X,0.73,"$[0.37$, $1.09]$",4.14,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.19,"$[-0.09$, $0.46]$",1.64,.238
noun_F_verb_X,noun_F - verb_X,0.19,"$[-0.08$, $0.47]$",1.66,.227
noun_M_verb_X,noun_M - verb_X,0.00,"$[-0.27$, $0.28]$",0.03,> .999


In [23]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
15agxf, 40asxm
Removing those cases from the analysis.”


# Condition 6 of 72

`tmin0==0.1 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.58,1.83,71.44,0.22,.214,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,-1.0114102,0.8609476,0.1361278
noun_M,42,-1.0157614,0.8567596,0.1322009
verb_X,42,-0.8723053,0.8240753,0.1271576


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.01,"$[-1.28$, $-0.75]$",-7.66,< .001
noun_M,noun_M,-1.04,"$[-1.31$, $-0.78]$",-7.90,< .001
verb_X,verb_X,-0.87,"$[-1.14$, $-0.61]$",-6.62,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.03,"$[-0.21$, $0.27]$",0.31,.948
noun_F_verb_X,noun_F - verb_X,-0.14,"$[-0.38$, $0.10]$",-1.36,.366
noun_M_verb_X,noun_M - verb_X,-0.17,"$[-0.41$, $0.07]$",-1.67,.222


In [24]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
10mczf, 18skxm, 26mwxf, 27zgxf
Removing those cases from the analysis.”


# Condition 7 of 72

`tmin0==0.1 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.48,1.88,69.74,0.21,.610,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,1.0774069,0.904685,0.1412881
noun_M,40,0.9021337,1.053093,0.1665087
verb_X,40,1.0499630,1.190883,0.1882952


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.08,"$[0.74$, $1.41]$",6.43,< .001
noun_M,noun_M,0.98,"$[0.64$, $1.31]$",5.84,< .001
verb_X,verb_X,1.02,"$[0.68$, $1.35]$",6.07,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.10,"$[-0.14$, $0.34]$",0.97,.597
noun_F_verb_X,noun_F - verb_X,0.06,"$[-0.18$, $0.30]$",0.59,.826
noun_M_verb_X,noun_M - verb_X,-0.04,"$[-0.28$, $0.20]$",-0.38,.923


In [25]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
03mjkf, 05jwxf, 12jnxf, 22foxm, 23zdxf, 24jkxf, 34gsxm, 39mrxm
Removing those cases from the analysis.”


# Condition 8 of 72

`tmin0==0.1 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.13,1.73,55.46,0.38,.135,.020


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,38,-0.6989812,0.8945182,0.1451100
noun_M,38,-0.7811927,0.8623121,0.1398855
verb_X,37,-0.4974695,0.9977210,0.1640243


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.83,"$[-1.12$, $-0.54]$",-5.72,< .001
noun_M,noun_M,-0.88,"$[-1.17$, $-0.59]$",-6.07,< .001
verb_X,verb_X,-0.61,"$[-0.90$, $-0.32]$",-4.19,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.05,"$[-0.29$, $0.39]$",0.35,.933
noun_F_verb_X,noun_F - verb_X,-0.22,"$[-0.56$, $0.12]$",-1.58,.261
noun_M_verb_X,noun_M - verb_X,-0.27,"$[-0.61$, $0.07]$",-1.94,.137


In [26]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
09msxf, 39mrxm
Removing those cases from the analysis.”


# Condition 9 of 72

`tmin0==0.1 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.73,1.77,67.38,0.34,.468,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,3.929941,2.380400,0.3763743
noun_M,40,4.179711,2.108586,0.3333967
verb_X,40,4.031834,2.288530,0.3618483


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.08,"$[3.36$, $4.80]$",11.46,< .001
noun_M,noun_M,4.18,"$[3.46$, $4.90]$",11.73,< .001
verb_X,verb_X,4.03,"$[3.31$, $4.75]$",11.32,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.10,"$[-0.39$, $0.20]$",-0.78,.714
noun_F_verb_X,noun_F - verb_X,0.05,"$[-0.25$, $0.35]$",0.41,.913
noun_M_verb_X,noun_M - verb_X,0.15,"$[-0.15$, $0.45]$",1.19,.462


In [27]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 04mkkm, 07zrxf, 09msxf, 10mczf, 11kbxm, 14ktxf, 17makf, 23zdxf, 29jjxf, 30mszf, 32mjxf, 34gsxm, 37lpxm, 38slxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 10 of 72

`tmin0==0.1 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.34,1.85,29.61,0.31,.052,.005


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,26,-2.134184,3.498041,0.6860222
noun_M,22,-1.721739,2.894102,0.6170246
verb_X,24,-2.303803,3.378332,0.6895991


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.88,"$[-4.40$, $-1.36]$",-4.00,.001
noun_M,noun_M,-2.45,"$[-3.98$, $-0.93]$",-3.41,.003
verb_X,verb_X,-2.86,"$[-4.38$, $-1.33]$",-3.97,.001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.43,"$[-0.88$, $0.03]$",-2.30,.070
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.48$, $0.43]$",-0.13,.991
noun_M_verb_X,noun_M - verb_X,0.40,"$[-0.05$, $0.86]$",2.17,.092


In [28]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 11 of 72

`tmin0==0.1 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.04,1.91,78.18,0.78,.954,.000


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,4.964665,2.926827,0.4516192
noun_M,42,4.911975,2.758665,0.4256713
verb_X,42,4.950949,2.587426,0.3992485


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.96,"$[4.11$, $5.82]$",11.65,< .001
noun_M,noun_M,4.91,"$[4.05$, $5.77]$",11.53,< .001
verb_X,verb_X,4.95,"$[4.09$, $5.81]$",11.62,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.05,"$[-0.40$, $0.50]$",0.28,.958
noun_F_verb_X,noun_F - verb_X,0.01,"$[-0.44$, $0.46]$",0.07,.997
noun_M_verb_X,noun_M - verb_X,-0.04,"$[-0.49$, $0.41]$",-0.21,.977


In [29]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 07zrxf, 09msxf, 25abxf, 32mjxf, 37lpxm
Removing those cases from the analysis.”


# Condition 12 of 72

`tmin0==0.1 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.20,1.60,38.31,0.49,.304,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,29,-0.3562989,2.160323,0.4011619
noun_M,27,-0.1143332,2.088925,0.4020138
verb_X,29,-0.3698908,1.870764,0.3473922


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.61,"$[-1.42$, $0.19]$",-1.56,.130
noun_M,noun_M,-0.36,"$[-1.16$, $0.45]$",-0.90,.375
verb_X,verb_X,-0.41,"$[-1.21$, $0.40]$",-1.04,.308


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.26,"$[-0.69$, $0.17]$",-1.47,.316
noun_F_verb_X,noun_F - verb_X,-0.21,"$[-0.63$, $0.22]$",-1.16,.480
noun_M_verb_X,noun_M - verb_X,0.05,"$[-0.37$, $0.48]$",0.30,.951


In [30]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 13 of 72

`tmin0==0.19 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.25,1.77,70.67,0.36,.119,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,2.109523,1.813195,0.2831735
noun_M,41,1.902960,1.989802,0.3107548
verb_X,41,1.863499,1.904413,0.2974194


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.11,"$[1.51$, $2.71]$",7.09,< .001
noun_M,noun_M,1.90,"$[1.30$, $2.50]$",6.40,< .001
verb_X,verb_X,1.86,"$[1.26$, $2.46]$",6.27,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.21,"$[-0.09$, $0.50]$",1.66,.228
noun_F_verb_X,noun_F - verb_X,0.25,"$[-0.05$, $0.54]$",1.97,.126
noun_M_verb_X,noun_M - verb_X,0.04,"$[-0.26$, $0.34]$",0.32,.946


In [31]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
06pppm, 07zrxf, 11kbxm, 13jsxm, 19jgxf, 22foxm, 28pmsf, 29jjxf, 30mszf, 31ltxf, 32mjxf, 33kkxm, 36rwxm, 40asxm, 42aszm
Removing those cases from the analysis.”


# Condition 14 of 72

`tmin0==0.19 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.08,1.67,31.68,0.50,.342,.006


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,28,-0.6297689,1.830365,0.3459064
noun_M,31,-0.2940191,2.314373,0.4156736
verb_X,24,-0.7034579,1.670528,0.3409951


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.70,"$[-1.46$, $0.06]$",-1.91,.068
noun_M,noun_M,-0.76,"$[-1.52$, $0.00]$",-2.07,.049
verb_X,verb_X,-0.99,"$[-1.75$, $-0.23]$",-2.69,.013


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.06,"$[-0.44$, $0.56]$",0.28,.957
noun_F_verb_X,noun_F - verb_X,0.28,"$[-0.22$, $0.78]$",1.39,.357
noun_M_verb_X,noun_M - verb_X,0.23,"$[-0.27$, $0.73]$",1.11,.516


In [32]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
20jpxf
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 15 of 72

`tmin0==0.19 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,7.02,1.95,77.85,0.26,.002,.009


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,0.9304280,1.760343,0.2749194
noun_M,42,0.5349085,1.712355,0.2642221
verb_X,42,0.6656455,1.810383,0.2793482


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.93,"$[0.37$, $1.49]$",3.36,.002
noun_M,noun_M,0.53,"$[-0.03$, $1.08]$",1.90,.064
verb_X,verb_X,0.65,"$[0.09$, $1.21]$",2.34,.024


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.40,"$[0.14$, $0.67]$",3.65,.001
noun_F_verb_X,noun_F - verb_X,0.28,"$[0.02$, $0.55]$",2.55,.034
noun_M_verb_X,noun_M - verb_X,-0.12,"$[-0.39$, $0.14]$",-1.11,.512


In [33]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 06pppm, 13jsxm, 15agxf, 16olxm, 17makf, 21koxf, 29jjxf, 30mszf, 32mjxf, 34gsxm, 37lpxm, 38slxm, 40asxm, 41roxm, 42aszm
Removing those cases from the analysis.”


# Condition 16 of 72

`tmin0==0.19 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.27,1.67,41.79,0.64,.726,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,36,-1.838795,1.471569,0.2452614
noun_M,36,-1.736157,1.782889,0.2971482
verb_X,31,-1.703051,1.545000,0.2774902


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.83,"$[-2.46$, $-1.20]$",-5.93,< .001
noun_M,noun_M,-1.77,"$[-2.40$, $-1.15]$",-5.74,< .001
verb_X,verb_X,-1.68,"$[-2.31$, $-1.06]$",-5.45,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.06,"$[-0.55$, $0.43]$",-0.28,.957
noun_F_verb_X,noun_F - verb_X,-0.15,"$[-0.64$, $0.34]$",-0.73,.748
noun_M_verb_X,noun_M - verb_X,-0.09,"$[-0.58$, $0.40]$",-0.45,.896


In [34]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
20jpxf, 28pmsf, 30mszf, 32mjxf, 35mwxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 17 of 72

`tmin0==0.19 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.19,1.97,70.80,0.23,.309,.007


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,1.226726,0.9799077,0.1530359
noun_M,40,1.410520,0.8645160,0.1366920
verb_X,38,1.322386,0.7692757,0.1247930


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.23,"$[0.95$, $1.52]$",8.67,< .001
noun_M,noun_M,1.40,"$[1.12$, $1.69]$",9.86,< .001
verb_X,verb_X,1.33,"$[1.04$, $1.61]$",9.33,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.17,"$[-0.44$, $0.09]$",-1.54,.278
noun_F_verb_X,noun_F - verb_X,-0.09,"$[-0.36$, $0.17]$",-0.85,.674
noun_M_verb_X,noun_M - verb_X,0.08,"$[-0.19$, $0.34]$",0.69,.768


In [35]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
03mjkf, 05jwxf, 08pnxm, 10mczf, 25abxf, 26mwxf, 30mszf, 41roxm
Removing those cases from the analysis.”


# Condition 18 of 72

`tmin0==0.19 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.00,1.56,51.42,0.43,.358,.005


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.2746207,1.2952111,0.1998554
noun_M,39,-0.2474937,1.1567577,0.1852295
verb_X,36,-0.2368590,0.9773267,0.1628878


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.42,"$[-0.82$, $-0.03]$",-2.15,.037
noun_M,noun_M,-0.39,"$[-0.78$, $0.01]$",-1.96,.055
verb_X,verb_X,-0.24,"$[-0.63$, $0.16]$",-1.20,.237


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.04,"$[-0.37$, $0.30]$",-0.26,.963
noun_F_verb_X,noun_F - verb_X,-0.19,"$[-0.53$, $0.15]$",-1.33,.383
noun_M_verb_X,noun_M - verb_X,-0.15,"$[-0.49$, $0.19]$",-1.07,.536


In [36]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 06pppm, 09msxf, 10mczf, 17makf, 18skxm, 20jpxf, 21koxf, 22foxm, 26mwxf, 27zgxf, 35mwxm, 40asxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 19 of 72

`tmin0==0.19 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.20,1.95,54.54,0.26,.121,.006


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,36,1.0890722,1.269065,0.2115109
noun_M,40,0.9588217,1.307105,0.2066714
verb_X,35,1.2751517,1.682552,0.2844032


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.24,"$[0.68$, $1.80]$",4.52,< .001
noun_M,noun_M,1.01,"$[0.45$, $1.57]$",3.67,.001
verb_X,verb_X,1.25,"$[0.69$, $1.81]$",4.55,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.23,"$[-0.08$, $0.55]$",1.78,.186
noun_F_verb_X,noun_F - verb_X,-0.01,"$[-0.33$, $0.31]$",-0.08,.997
noun_M_verb_X,noun_M - verb_X,-0.24,"$[-0.56$, $0.07]$",-1.86,.161


In [37]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
24jkxf, 42aszm
Removing those cases from the analysis.”


# Condition 20 of 72

`tmin0==0.19 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.87,1.74,66.27,0.25,.071,.009


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,-0.5738661,1.042069,0.1627438
noun_M,39,-0.7602025,1.293015,0.2070482
verb_X,39,-0.5101953,1.174297,0.1880380


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.59,"$[-0.96$, $-0.22]$",-3.24,.002
noun_M,noun_M,-0.76,"$[-1.13$, $-0.39]$",-4.17,< .001
verb_X,verb_X,-0.51,"$[-0.88$, $-0.14]$",-2.80,.007


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.17,"$[-0.09$, $0.42]$",1.58,.259
noun_F_verb_X,noun_F - verb_X,-0.08,"$[-0.34$, $0.17]$",-0.76,.727
noun_M_verb_X,noun_M - verb_X,-0.25,"$[-0.50$, $0.00]$",-2.35,.055


In [38]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
03mjkf, 06pppm, 07zrxf, 08pnxm, 11kbxm, 13jsxm, 14ktxf, 17makf, 32mjxf, 35mwxm, 37lpxm, 38slxm, 42aszm
Removing those cases from the analysis.”


# Condition 21 of 72

`tmin0==0.19 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.24,1.64,42.71,0.49,.027,.007


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,36,0.9028009,2.610894,0.435149
noun_M,37,1.0823633,2.259017,0.371380
verb_X,33,0.4956733,2.366065,0.411879


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.59,"$[-0.41$, $1.59]$",1.21,.235
noun_M,noun_M,1.08,"$[0.08$, $2.08]$",2.22,.035
verb_X,verb_X,0.92,"$[-0.08$, $1.92]$",1.88,.071


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.49,"$[-0.91$, $-0.08]$",-2.86,.016
noun_F_verb_X,noun_F - verb_X,-0.33,"$[-0.74$, $0.09]$",-1.89,.151
noun_M_verb_X,noun_M - verb_X,0.17,"$[-0.25$, $0.58]$",0.97,.599


In [39]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
01mkxf, 24jkxf, 30mszf, 35mwxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 22 of 72

`tmin0==0.19 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,9.90,1.93,69.42,0.33,< .001,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,39,-2.393996,2.613687,0.4185249
noun_M,40,-1.670696,2.592275,0.4098747
verb_X,38,-2.064230,2.688970,0.4362086


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.44,"$[-3.32$, $-1.56]$",-5.60,< .001
noun_M,noun_M,-1.86,"$[-2.74$, $-0.98]$",-4.27,< .001
verb_X,verb_X,-2.13,"$[-3.01$, $-1.24]$",-4.88,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.58,"$[-0.89$, $-0.27]$",-4.44,< .001
noun_F_verb_X,noun_F - verb_X,-0.32,"$[-0.63$, $0.00]$",-2.42,.047
noun_M_verb_X,noun_M - verb_X,0.26,"$[-0.05$, $0.58]$",2.02,.114


In [40]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
06pppm, 07zrxf, 16olxm, 17makf, 21koxf, 25abxf, 26mwxf, 29jjxf, 32mjxf, 36rwxm, 37lpxm, 38slxm, 40asxm, 41roxm, 42aszm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 23 of 72

`tmin0==0.19 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.53,1.99,47.66,0.43,.590,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,35,2.562109,1.843902,0.3116763
noun_M,36,2.782443,2.809274,0.4682124
verb_X,30,2.415413,1.933609,0.3530270


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.52,"$[1.68$, $3.36]$",6.14,< .001
noun_M,noun_M,2.70,"$[1.86$, $3.55]$",6.59,< .001
verb_X,verb_X,2.65,"$[1.81$, $3.49]$",6.46,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.19,"$[-0.63$, $0.26]$",-1.00,.578
noun_F_verb_X,noun_F - verb_X,-0.13,"$[-0.58$, $0.32]$",-0.70,.762
noun_M_verb_X,noun_M - verb_X,0.06,"$[-0.39$, $0.50]$",0.30,.952


In [41]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
26mwxf
Removing those cases from the analysis.”


# Condition 24 of 72

`tmin0==0.19 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.24,1.49,58.15,0.46,.720,.000


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,-0.3429737,2.412656,0.3767935
noun_M,41,-0.2612989,2.349352,0.3669072
verb_X,40,-0.3024858,2.327760,0.3680512


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.34,"$[-1.10$, $0.43]$",-0.89,.379
noun_M,noun_M,-0.25,"$[-1.01$, $0.52]$",-0.65,.519
verb_X,verb_X,-0.30,"$[-1.06$, $0.46]$",-0.80,.427


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.09,"$[-0.40$, $0.22]$",-0.69,.772
noun_F_verb_X,noun_F - verb_X,-0.03,"$[-0.35$, $0.28]$",-0.25,.966
noun_M_verb_X,noun_M - verb_X,0.06,"$[-0.26$, $0.37]$",0.44,.900


In [42]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
06pppm, 17makf, 30mszf
Removing those cases from the analysis.”


# Condition 25 of 72

`tmin0==0.35 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.74,1.88,71.25,0.47,.075,.009


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,-1.224847,1.593235,0.2488215
noun_M,42,-1.248104,1.850578,0.2855504
verb_X,40,-1.540407,1.500329,0.2372229


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.17,"$[-1.66$, $-0.68]$",-4.76,< .001
noun_M,noun_M,-1.11,"$[-1.61$, $-0.62]$",-4.52,< .001
verb_X,verb_X,-1.44,"$[-1.93$, $-0.95]$",-5.85,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.06,"$[-0.42$, $0.30]$",-0.39,.920
noun_F_verb_X,noun_F - verb_X,0.27,"$[-0.09$, $0.63]$",1.81,.175
noun_M_verb_X,noun_M - verb_X,0.33,"$[-0.03$, $0.69]$",2.19,.079


In [43]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
08pnxm, 18skxm, 23zdxf, 32mjxf, 38slxm
Removing those cases from the analysis.”


# Condition 26 of 72

`tmin0==0.35 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.32,1.88,67.67,0.48,.109,.007


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,-2.655767,1.755156,0.2775146
noun_M,41,-2.890340,1.663849,0.2598496
verb_X,38,-2.708688,1.643969,0.2666869


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.69,"$[-3.24$, $-2.14]$",-9.82,< .001
noun_M,noun_M,-3.01,"$[-3.56$, $-2.46]$",-11.00,< .001
verb_X,verb_X,-2.76,"$[-3.31$, $-2.21]$",-10.09,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.32,"$[-0.05$, $0.70]$",2.06,.106
noun_F_verb_X,noun_F - verb_X,0.07,"$[-0.30$, $0.45]$",0.47,.888
noun_M_verb_X,noun_M - verb_X,-0.25,"$[-0.62$, $0.13]$",-1.59,.256


In [44]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 27 of 72

`tmin0==0.35 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.18,1.99,81.43,0.40,.120,.006


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.873510,1.380679,0.2130434
noun_M,42,-2.133609,1.487869,0.2295832
verb_X,42,-1.896576,1.563693,0.2412831


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.87,"$[-2.33$, $-1.42]$",-8.21,< .001
noun_M,noun_M,-2.13,"$[-2.59$, $-1.68]$",-9.35,< .001
verb_X,verb_X,-1.90,"$[-2.35$, $-1.44]$",-8.31,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.26,"$[-0.07$, $0.59]$",1.88,.150
noun_F_verb_X,noun_F - verb_X,0.02,"$[-0.31$, $0.35]$",0.17,.985
noun_M_verb_X,noun_M - verb_X,-0.24,"$[-0.57$, $0.09]$",-1.72,.205


In [45]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
08pnxm, 30mszf, 32mjxf, 34gsxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 28 of 72

`tmin0==0.35 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.07,1.94,69.93,0.47,.022,.011


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,-3.321413,1.684845,0.2631286
noun_M,39,-3.596182,1.632623,0.2614290
verb_X,38,-3.188799,1.872108,0.3036961


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.44,"$[-4.02$, $-2.86]$",-11.99,< .001
noun_M,noun_M,-3.65,"$[-4.23$, $-3.07]$",-12.72,< .001
verb_X,verb_X,-3.21,"$[-3.78$, $-2.63]$",-11.17,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.21,"$[-0.16$, $0.58]$",1.35,.375
noun_F_verb_X,noun_F - verb_X,-0.24,"$[-0.61$, $0.14]$",-1.51,.295
noun_M_verb_X,noun_M - verb_X,-0.45,"$[-0.82$, $-0.07]$",-2.85,.016


In [46]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
05jwxf, 07zrxf, 41roxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 29 of 72

`tmin0==0.35 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.38,1.98,75.09,0.28,.681,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.535496,1.382349,0.2133011
noun_M,42,1.471160,1.198128,0.1848752
verb_X,39,1.583289,1.173459,0.1879038


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.66,"$[1.27$, $2.05]$",8.59,< .001
noun_M,noun_M,1.56,"$[1.17$, $1.95]$",8.07,< .001
verb_X,verb_X,1.58,"$[1.19$, $1.97]$",8.19,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.10,"$[-0.19$, $0.38]$",0.83,.683
noun_F_verb_X,noun_F - verb_X,0.08,"$[-0.21$, $0.36]$",0.64,.797
noun_M_verb_X,noun_M - verb_X,-0.02,"$[-0.31$, $0.26]$",-0.19,.980


In [47]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf, 38slxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 30 of 72

`tmin0==0.35 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.39,1.98,77.08,0.23,.256,.004


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.3951065,1.143779,0.1764889
noun_M,42,0.3698103,1.253913,0.1934829
verb_X,40,0.5942835,1.175542,0.1858695


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.45,"$[0.08$, $0.83]$",2.43,.019
noun_M,noun_M,0.43,"$[0.05$, $0.80]$",2.29,.026
verb_X,verb_X,0.59,"$[0.22$, $0.97]$",3.18,.003


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.03,"$[-0.23$, $0.28]$",0.24,.969
noun_F_verb_X,noun_F - verb_X,-0.14,"$[-0.40$, $0.12]$",-1.31,.396
noun_M_verb_X,noun_M - verb_X,-0.17,"$[-0.42$, $0.09]$",-1.55,.275


In [48]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
38slxm, 40asxm, 42aszm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 31 of 72

`tmin0==0.35 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.03,1.91,72.49,0.31,.360,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,1.577825,1.622378,0.2565204
noun_M,42,1.516462,1.361944,0.2101525
verb_X,41,1.651723,1.570619,0.2452895


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.59,"$[1.10$, $2.09]$",6.49,< .001
noun_M,noun_M,1.47,"$[0.97$, $1.96]$",5.98,< .001
verb_X,verb_X,1.64,"$[1.15$, $2.14]$",6.69,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.12,"$[-0.17$, $0.42]$",0.99,.584
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.35$, $0.25]$",-0.40,.915
noun_M_verb_X,noun_M - verb_X,-0.17,"$[-0.47$, $0.12]$",-1.39,.349


In [49]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
37lpxm, 40asxm
Removing those cases from the analysis.”


# Condition 32 of 72

`tmin0==0.35 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,6.51,1.73,67.65,0.33,.004,.015


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.6485346,1.508217,0.2327229
noun_M,42,0.3911677,1.642567,0.2534536
verb_X,40,0.7255953,1.484348,0.2346960


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.56,"$[0.10$, $1.03]$",2.43,.019
noun_M,noun_M,0.30,"$[-0.17$, $0.76]$",1.28,.207
verb_X,verb_X,0.73,"$[0.26$, $1.19]$",3.13,.003


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.27,"$[-0.02$, $0.55]$",2.22,.073
noun_F_verb_X,noun_F - verb_X,-0.16,"$[-0.45$, $0.12]$",-1.35,.372
noun_M_verb_X,noun_M - verb_X,-0.43,"$[-0.72$, $-0.14]$",-3.57,.002


In [50]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
08pnxm, 28pmsf, 32mjxf, 34gsxm, 35mwxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 33 of 72

`tmin0==0.35 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.07,1.96,68.51,0.49,.054,.005


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,40,2.498768,2.525809,0.3993655
noun_M,38,3.057710,2.177233,0.3531938
verb_X,39,2.630638,2.368774,0.3793073


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.55,"$[1.74$, $3.35]$",6.41,< .001
noun_M,noun_M,2.94,"$[2.13$, $3.74]$",7.39,< .001
verb_X,verb_X,2.65,"$[1.84$, $3.45]$",6.67,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.39,"$[-0.78$, $0.00]$",-2.39,.051
noun_F_verb_X,noun_F - verb_X,-0.10,"$[-0.49$, $0.29]$",-0.62,.809
noun_M_verb_X,noun_M - verb_X,0.29,"$[-0.10$, $0.68]$",1.77,.188


In [51]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
20jpxf, 42aszm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 34 of 72

`tmin0==0.35 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.25,1.99,77.74,0.49,.044,.006


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.153308,2.085355,0.3217772
noun_M,41,1.505719,1.887321,0.2947501
verb_X,40,1.432062,2.135348,0.3376282


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.11,"$[0.45$, $1.76]$",3.41,.001
noun_M,noun_M,1.47,"$[0.82$, $2.13]$",4.53,< .001
verb_X,verb_X,1.43,"$[0.78$, $2.09]$",4.40,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.36,"$[-0.74$, $0.01]$",-2.32,.058
noun_F_verb_X,noun_F - verb_X,-0.32,"$[-0.70$, $0.05]$",-2.07,.103
noun_M_verb_X,noun_M - verb_X,0.04,"$[-0.33$, $0.41]$",0.25,.965


In [52]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
18skxm, 23zdxf, 25abxf, 26mwxf, 30mszf, 32mjxf, 38slxm
Removing those cases from the analysis.”


# Condition 35 of 72

`tmin0==0.35 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.95,1.83,62.10,0.39,.155,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,37,3.822664,2.279750,0.3747885
noun_M,41,4.319591,2.038841,0.3184134
verb_X,39,4.129674,2.197261,0.3518434


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.91,"$[3.16$, $4.67]$",10.49,< .001
noun_M,noun_M,4.18,"$[3.42$, $4.93]$",11.20,< .001
verb_X,verb_X,3.97,"$[3.22$, $4.73]$",10.65,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.27,"$[-0.61$, $0.07]$",-1.88,.152
noun_F_verb_X,noun_F - verb_X,-0.06,"$[-0.40$, $0.28]$",-0.42,.906
noun_M_verb_X,noun_M - verb_X,0.21,"$[-0.13$, $0.55]$",1.46,.318


In [53]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
06pppm, 09msxf, 20jpxf, 22foxm, 41roxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 36 of 72

`tmin0==0.35 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.11,1.93,69.46,0.43,.332,.002


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,2.389840,2.183137,0.3409487
noun_M,40,2.451460,2.365666,0.3740446
verb_X,39,2.684616,2.116706,0.3389443


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.46,"$[1.73$, $3.20]$",6.75,< .001
noun_M,noun_M,2.65,"$[1.92$, $3.39]$",7.27,< .001
verb_X,verb_X,2.66,"$[1.92$, $3.40]$",7.29,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.19,"$[-0.55$, $0.17]$",-1.27,.418
noun_F_verb_X,noun_F - verb_X,-0.20,"$[-0.55$, $0.16]$",-1.32,.391
noun_M_verb_X,noun_M - verb_X,-0.01,"$[-0.36$, $0.35]$",-0.05,.999


In [54]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 37 of 72

`tmin0==0.41 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.83,1.66,67.97,1.09,.422,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.9826644,2.376311,0.3666727
noun_M,42,-1.2303364,1.812512,0.2796767
verb_X,42,-1.1917032,2.108405,0.3253340


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.98,"$[-1.64$, $-0.33]$",-3.02,.004
noun_M,noun_M,-1.23,"$[-1.88$, $-0.58]$",-3.78,< .001
verb_X,verb_X,-1.19,"$[-1.84$, $-0.54]$",-3.66,.001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.25,"$[-0.25$, $0.74]$",1.19,.460
noun_F_verb_X,noun_F - verb_X,0.21,"$[-0.29$, $0.70]$",1.01,.574
noun_M_verb_X,noun_M - verb_X,-0.04,"$[-0.53$, $0.46]$",-0.19,.981


In [55]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 38 of 72

`tmin0==0.41 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.38,1.89,77.40,0.78,.102,.006


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-3.020868,2.226987,0.3436316
noun_M,42,-3.309663,1.924940,0.2970248
verb_X,42,-2.915711,2.270142,0.3502906


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.02,"$[-3.69$, $-2.36]$",-9.12,< .001
noun_M,noun_M,-3.31,"$[-3.97$, $-2.64]$",-9.99,< .001
verb_X,verb_X,-2.92,"$[-3.58$, $-2.25]$",-8.80,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.29,"$[-0.16$, $0.73]$",1.55,.275
noun_F_verb_X,noun_F - verb_X,-0.11,"$[-0.55$, $0.34]$",-0.56,.840
noun_M_verb_X,noun_M - verb_X,-0.39,"$[-0.84$, $0.05]$",-2.11,.094


In [56]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 39 of 72

`tmin0==0.41 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.97,1.92,76.66,0.35,.148,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.042816,2.601125,0.4013623
noun_M,42,-1.435159,1.854583,0.2861684
verb_X,41,-1.577668,1.938928,0.3028098


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.34,"$[-1.91$, $-0.78]$",-4.76,< .001
noun_M,noun_M,-1.55,"$[-2.12$, $-0.98]$",-5.48,< .001
verb_X,verb_X,-1.58,"$[-2.15$, $-1.01]$",-5.59,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.20,"$[-0.10$, $0.51]$",1.59,.256
noun_F_verb_X,noun_F - verb_X,0.23,"$[-0.07$, $0.54]$",1.83,.167
noun_M_verb_X,noun_M - verb_X,0.03,"$[-0.28$, $0.34]$",0.24,.970


In [57]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf
Removing those cases from the analysis.”


# Condition 40 of 72

`tmin0==0.41 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.23,1.60,64.11,0.52,.126,.004


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,-3.594184,1.767796,0.2760833
noun_M,42,-3.721401,1.868240,0.2882757
verb_X,41,-3.458458,1.945652,0.3038598


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.59,"$[-4.18$, $-3.01]$",-12.35,< .001
noun_M,noun_M,-3.76,"$[-4.35$, $-3.17]$",-12.92,< .001
verb_X,verb_X,-3.46,"$[-4.04$, $-2.87]$",-11.88,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.17,"$[-0.18$, $0.51]$",1.16,.480
noun_F_verb_X,noun_F - verb_X,-0.14,"$[-0.48$, $0.21]$",-0.95,.612
noun_M_verb_X,noun_M - verb_X,-0.30,"$[-0.64$, $0.04]$",-2.11,.094


In [58]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 41 of 72

`tmin0==0.41 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.74,1.57,64.55,0.39,.452,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.518711,1.542091,0.2379498
noun_M,42,1.663074,1.226860,0.1893086
verb_X,42,1.566396,1.368801,0.2112106


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.52,"$[1.09$, $1.95]$",7.11,< .001
noun_M,noun_M,1.66,"$[1.23$, $2.09]$",7.78,< .001
verb_X,verb_X,1.57,"$[1.14$, $2.00]$",7.33,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.14,"$[-0.43$, $0.14]$",-1.19,.461
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.34$, $0.24]$",-0.39,.918
noun_M_verb_X,noun_M - verb_X,0.10,"$[-0.19$, $0.39]$",0.80,.705


In [59]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 42 of 72

`tmin0==0.41 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.54,1.57,64.27,0.38,.543,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.1822789,1.604342,0.2475554
noun_M,42,0.1850149,1.333468,0.2057585
verb_X,42,0.2908507,1.600486,0.2469604


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.18,"$[-0.29$, $0.65]$",0.78,.440
noun_M,noun_M,0.19,"$[-0.29$, $0.66]$",0.79,.433
verb_X,verb_X,0.29,"$[-0.18$, $0.76]$",1.24,.220


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.00,"$[-0.29$, $0.28]$",-0.02,> .999
noun_F_verb_X,noun_F - verb_X,-0.11,"$[-0.39$, $0.18]$",-0.91,.635
noun_M_verb_X,noun_M - verb_X,-0.11,"$[-0.39$, $0.18]$",-0.89,.650


In [60]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 43 of 72

`tmin0==0.41 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.68,1.72,70.61,0.43,.487,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.800638,1.554869,0.2399215
noun_M,42,1.692684,1.450329,0.2237906
verb_X,42,1.650715,1.528723,0.2358871


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.80,"$[1.33$, $2.27]$",7.72,< .001
noun_M,noun_M,1.69,"$[1.22$, $2.16]$",7.26,< .001
verb_X,verb_X,1.65,"$[1.18$, $2.12]$",7.08,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.11,"$[-0.21$, $0.42]$",0.82,.694
noun_F_verb_X,noun_F - verb_X,0.15,"$[-0.17$, $0.47]$",1.13,.496
noun_M_verb_X,noun_M - verb_X,0.04,"$[-0.27$, $0.36]$",0.32,.946


In [61]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 44 of 72

`tmin0==0.41 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.44,1.72,70.44,0.35,.044,.007


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.3720706,1.635512,0.2523650
noun_M,42,0.1969286,1.480558,0.2284550
verb_X,42,0.5122825,1.558251,0.2404434


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.37,"$[-0.11$, $0.86]$",1.55,.129
noun_M,noun_M,0.20,"$[-0.29$, $0.68]$",0.82,.417
verb_X,verb_X,0.51,"$[0.03$, $1.00]$",2.13,.038


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.18,"$[-0.11$, $0.46]$",1.45,.318
noun_F_verb_X,noun_F - verb_X,-0.14,"$[-0.43$, $0.15]$",-1.16,.478
noun_M_verb_X,noun_M - verb_X,-0.32,"$[-0.60$, $-0.03]$",-2.62,.028


In [62]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf, 30mszf
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 45 of 72

`tmin0==0.41 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.94,1.94,75.59,0.45,.391,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,2.571274,2.532714,0.3955435
noun_M,41,2.823260,2.204071,0.3442181
verb_X,41,2.658914,2.369363,0.3700323


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.67,"$[1.92$, $3.42]$",7.16,< .001
noun_M,noun_M,2.86,"$[2.11$, $3.62]$",7.69,< .001
verb_X,verb_X,2.73,"$[1.98$, $3.48]$",7.32,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.20,"$[-0.55$, $0.15]$",-1.34,.375
noun_F_verb_X,noun_F - verb_X,-0.06,"$[-0.41$, $0.29]$",-0.42,.905
noun_M_verb_X,noun_M - verb_X,0.14,"$[-0.22$, $0.49]$",0.92,.630


In [63]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf, 30mszf
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 46 of 72

`tmin0==0.41 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.00,2.00,77.93,0.42,.009,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.07821053,2.916449,0.4500178
noun_M,41,0.52855269,1.981732,0.3094946
verb_X,41,0.65966110,2.358293,0.3683034


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.32,"$[-0.34$, $0.99]$",0.98,.334
noun_M,noun_M,0.62,"$[-0.05$, $1.29]$",1.87,.068
verb_X,verb_X,0.77,"$[0.11$, $1.44]$",2.34,.024


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.30,"$[-0.64$, $0.05]$",-2.04,.110
noun_F_verb_X,noun_F - verb_X,-0.45,"$[-0.80$, $-0.11]$",-3.11,.007
noun_M_verb_X,noun_M - verb_X,-0.16,"$[-0.50$, $0.19]$",-1.08,.531


In [64]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf
Removing those cases from the analysis.”


# Condition 47 of 72

`tmin0==0.41 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.47,1.83,73.18,0.53,.096,.005


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,41,3.860333,1.924411,0.3005425
noun_M,42,4.078567,2.254974,0.3479500
verb_X,41,3.973052,2.113261,0.3300359


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,3.86,"$[3.21$, $4.51]$",11.97,< .001
noun_M,noun_M,4.20,"$[3.55$, $4.84]$",13.01,< .001
verb_X,verb_X,3.97,"$[3.32$, $4.62]$",12.32,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.33,"$[-0.70$, $0.03]$",-2.18,.080
noun_F_verb_X,noun_F - verb_X,-0.11,"$[-0.48$, $0.25]$",-0.73,.744
noun_M_verb_X,noun_M - verb_X,0.22,"$[-0.14$, $0.59]$",1.45,.321


In [65]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf
Removing those cases from the analysis.”


# Condition 48 of 72

`tmin0==0.41 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.37,1.91,76.25,0.55,.682,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.367303,3.032355,0.4679025
noun_M,42,1.671950,2.297869,0.3545688
verb_X,41,1.833801,2.239861,0.3498075


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.70,"$[1.00$, $2.39]$",4.90,< .001
noun_M,noun_M,1.77,"$[1.08$, $2.47]$",5.12,< .001
verb_X,verb_X,1.83,"$[1.14$, $2.53]$",5.30,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.08,"$[-0.46$, $0.31]$",-0.49,.878
noun_F_verb_X,noun_F - verb_X,-0.14,"$[-0.52$, $0.25]$",-0.86,.668
noun_M_verb_X,noun_M - verb_X,-0.06,"$[-0.44$, $0.32]$",-0.37,.927


In [66]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 49 of 72

`tmin0==0.6 & chan0=="LF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.55,1.84,75.37,0.59,.016,.008


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.0912757,2.386567,0.3682553
noun_M,42,-0.3545464,2.099339,0.3239351
verb_X,42,-0.5763876,1.999215,0.3084856


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.09,"$[-0.76$, $0.58]$",-0.27,.786
noun_M,noun_M,-0.35,"$[-1.03$, $0.32]$",-1.06,.295
verb_X,verb_X,-0.58,"$[-1.25$, $0.10]$",-1.72,.091


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.26,"$[-0.12$, $0.65]$",1.64,.237
noun_F_verb_X,noun_F - verb_X,0.49,"$[0.10$, $0.87]$",3.01,.010
noun_M_verb_X,noun_M - verb_X,0.22,"$[-0.16$, $0.61]$",1.38,.357


In [67]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 50 of 72

`tmin0==0.6 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.47,1.86,76.13,0.55,.236,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.268396,1.968912,0.3038097
noun_M,42,-2.534242,1.813587,0.2798426
verb_X,42,-2.378760,1.602524,0.2472749


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.27,"$[-2.83$, $-1.71]$",-8.16,< .001
noun_M,noun_M,-2.53,"$[-3.09$, $-1.98]$",-9.12,< .001
verb_X,verb_X,-2.38,"$[-2.94$, $-1.82]$",-8.56,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.27,"$[-0.11$, $0.64]$",1.71,.208
noun_F_verb_X,noun_F - verb_X,0.11,"$[-0.26$, $0.48]$",0.71,.759
noun_M_verb_X,noun_M - verb_X,-0.16,"$[-0.53$, $0.22]$",-1.00,.579


In [68]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 51 of 72

`tmin0==0.6 & chan0=="RF" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.54,1.79,73.55,0.54,.007,.009


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.540771,2.315286,0.3572564
noun_M,42,1.074418,1.969072,0.3038344
verb_X,42,1.137772,2.222424,0.3429274


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.54,"$[0.87$, $2.22]$",4.59,< .001
noun_M,noun_M,1.07,"$[0.40$, $1.75]$",3.20,.002
verb_X,verb_X,1.14,"$[0.46$, $1.81]$",3.39,.001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.47,"$[0.10$, $0.83]$",3.07,.008
noun_F_verb_X,noun_F - verb_X,0.40,"$[0.04$, $0.77]$",2.65,.026
noun_M_verb_X,noun_M - verb_X,-0.06,"$[-0.43$, $0.30]$",-0.42,.909


In [69]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 52 of 72

`tmin0==0.6 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.20,1.99,81.57,0.55,.018,.010


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-1.233521,2.007828,0.3098146
noun_M,42,-1.613235,1.869096,0.2884077
verb_X,42,-1.186201,1.888123,0.2913437


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-1.23,"$[-1.83$, $-0.64]$",-4.16,< .001
noun_M,noun_M,-1.61,"$[-2.21$, $-1.02]$",-5.44,< .001
verb_X,verb_X,-1.19,"$[-1.78$, $-0.59]$",-4.00,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.38,"$[-0.01$, $0.77]$",2.35,.054
noun_F_verb_X,noun_F - verb_X,-0.05,"$[-0.43$, $0.34]$",-0.29,.954
noun_M_verb_X,noun_M - verb_X,-0.43,"$[-0.81$, $-0.04]$",-2.64,.026


In [70]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 53 of 72

`tmin0==0.6 & chan0=="LC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.88,1.96,80.31,0.22,.160,.004


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,1.1129694,1.299208,0.2004721
noun_M,42,0.9897037,1.302126,0.2009223
verb_X,42,0.9202174,1.407766,0.2172230


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,1.11,"$[0.70$, $1.53]$",5.39,< .001
noun_M,noun_M,0.99,"$[0.57$, $1.40]$",4.80,< .001
verb_X,verb_X,0.92,"$[0.51$, $1.34]$",4.46,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.12,"$[-0.12$, $0.36]$",1.22,.443
noun_F_verb_X,noun_F - verb_X,0.19,"$[-0.05$, $0.43]$",1.91,.141
noun_M_verb_X,noun_M - verb_X,0.07,"$[-0.17$, $0.31]$",0.69,.770


In [71]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 54 of 72

`tmin0==0.6 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.48,1.79,73.28,0.32,.041,.007


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.6616578,1.486804,0.2294189
noun_M,42,-0.7288810,1.366429,0.2108446
verb_X,42,-0.4370582,1.522761,0.2349672


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.66,"$[-1.11$, $-0.21]$",-2.94,.005
noun_M,noun_M,-0.73,"$[-1.18$, $-0.28]$",-3.23,.002
verb_X,verb_X,-0.44,"$[-0.89$, $0.02]$",-1.94,.058


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.07,"$[-0.21$, $0.34]$",0.58,.831
noun_F_verb_X,noun_F - verb_X,-0.22,"$[-0.50$, $0.05]$",-1.94,.135
noun_M_verb_X,noun_M - verb_X,-0.29,"$[-0.57$, $-0.02]$",-2.52,.036


In [72]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 55 of 72

`tmin0==0.6 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.98,1.84,75.52,0.37,.061,.010


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.268632,1.279591,0.1974452
noun_M,42,1.957888,1.315877,0.2030443
verb_X,42,2.143979,1.355660,0.2091829


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.27,"$[1.86$, $2.68]$",11.16,< .001
noun_M,noun_M,1.96,"$[1.55$, $2.37]$",9.63,< .001
verb_X,verb_X,2.14,"$[1.74$, $2.55]$",10.55,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.31,"$[0.01$, $0.62]$",2.43,.045
noun_F_verb_X,noun_F - verb_X,0.12,"$[-0.18$, $0.43]$",0.97,.595
noun_M_verb_X,noun_M - verb_X,-0.19,"$[-0.49$, $0.12]$",-1.45,.319


In [73]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 56 of 72

`tmin0==0.6 & chan0=="RC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,9.63,1.66,68.24,0.44,< .001,.025


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.5139280,1.528124,0.2357947
noun_M,42,0.2874146,1.429243,0.2205369
verb_X,42,0.8610185,1.486238,0.2293315


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.51,"$[0.06$, $0.97]$",2.25,.029
noun_M,noun_M,0.29,"$[-0.17$, $0.75]$",1.26,.214
verb_X,verb_X,0.86,"$[0.40$, $1.32]$",3.77,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.23,"$[-0.09$, $0.54]$",1.72,.204
noun_F_verb_X,noun_F - verb_X,-0.35,"$[-0.66$, $-0.03]$",-2.64,.027
noun_M_verb_X,noun_M - verb_X,-0.57,"$[-0.89$, $-0.26]$",-4.36,< .001


In [74]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 57 of 72

`tmin0==0.6 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.43,1.95,79.92,0.60,.646,.001


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,0.4781189,2.034312,0.3139011
noun_M,42,0.5997088,2.145143,0.3310027
verb_X,42,0.4562686,2.041356,0.3149881


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.48,"$[-0.16$, $1.12]$",1.49,.142
noun_M,noun_M,0.60,"$[-0.04$, $1.24]$",1.87,.067
verb_X,verb_X,0.46,"$[-0.19$, $1.10]$",1.43,.160


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.12,"$[-0.52$, $0.28]$",-0.73,.746
noun_F_verb_X,noun_F - verb_X,0.02,"$[-0.38$, $0.42]$",0.13,.991
noun_M_verb_X,noun_M - verb_X,0.14,"$[-0.25$, $0.54]$",0.86,.665


In [75]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 58 of 72

`tmin0==0.6 & chan0=="LP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,10.39,1.90,78.00,0.50,< .001,.014


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-2.114307,2.481554,0.3829120
noun_M,42,-1.707023,2.214557,0.3417136
verb_X,42,-1.432424,2.500919,0.3859002


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-2.11,"$[-2.86$, $-1.37]$",-5.70,< .001
noun_M,noun_M,-1.71,"$[-2.45$, $-0.96]$",-4.60,< .001
verb_X,verb_X,-1.43,"$[-2.18$, $-0.69]$",-3.86,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.41,"$[-0.77$, $-0.05]$",-2.71,.022
noun_F_verb_X,noun_F - verb_X,-0.68,"$[-1.04$, $-0.32]$",-4.53,< .001
noun_M_verb_X,noun_M - verb_X,-0.27,"$[-0.63$, $0.08]$",-1.82,.168


In [76]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 59 of 72

`tmin0==0.6 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.04,1.87,76.84,0.52,.140,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,2.213899,2.161625,0.3335459
noun_M,42,2.490893,2.230421,0.3441615
verb_X,42,2.233618,1.867128,0.2881041


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,2.21,"$[1.56$, $2.86]$",6.86,< .001
noun_M,noun_M,2.49,"$[1.84$, $3.14]$",7.72,< .001
verb_X,verb_X,2.23,"$[1.58$, $2.88]$",6.92,< .001


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.28,"$[-0.64$, $0.09]$",-1.81,.173
noun_F_verb_X,noun_F - verb_X,-0.02,"$[-0.39$, $0.35]$",-0.13,.991
noun_M_verb_X,noun_M - verb_X,0.26,"$[-0.11$, $0.62]$",1.68,.219


In [77]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 60 of 72

`tmin0==0.6 & chan0=="RP" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.55,1.76,72.06,0.69,.017,.006


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,42,-0.1405336,2.936157,0.4530589
noun_M,42,0.2013572,2.507150,0.3868617
verb_X,42,0.3626507,2.572085,0.3968813


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.14,"$[-0.97$, $0.69]$",-0.34,.735
noun_M,noun_M,0.20,"$[-0.63$, $1.03]$",0.49,.628
verb_X,verb_X,0.36,"$[-0.47$, $1.19]$",0.88,.385


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.34,"$[-0.75$, $0.06]$",-2.01,.117
noun_F_verb_X,noun_F - verb_X,-0.50,"$[-0.91$, $-0.10]$",-2.95,.011
noun_M_verb_X,noun_M - verb_X,-0.16,"$[-0.57$, $0.25]$",-0.95,.613


In [78]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
19jgxf, 24jkxf, 25abxf, 30mszf, 31ltxf, 33kkxm, 39mrxm
Removing those cases from the analysis.”


ERROR: Error: No observations remain after removing missing values.
  Try adding to ANOVA call: na.rm = TRUE


In [79]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
09msxf, 10mczf, 13jsxm, 15agxf, 22foxm, 25abxf, 32mjxf, 36rwxm, 37lpxm, 39mrxm, 40asxm
Removing those cases from the analysis.”


# Condition 62 of 72

`tmin0==0.11 & chan0=="LF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.46,1.23,3.68,0.45,.575,.035


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,7,-2.546522,1.065947,0.4028901
noun_M,10,-2.645662,1.143593,0.3616357
verb_X,9,-2.037640,1.683528,0.5611759


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.14,"$[-4.30$, $-1.98]$",-7.03,.001
noun_M,noun_M,-2.83,"$[-3.99$, $-1.67]$",-6.34,.002
verb_X,verb_X,-2.82,"$[-3.98$, $-1.67]$",-6.32,.002


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.31,"$[-1.45$, $0.84]$",-0.82,.705
noun_F_verb_X,noun_F - verb_X,-0.31,"$[-1.46$, $0.83]$",-0.85,.690
noun_M_verb_X,noun_M - verb_X,-0.01,"$[-1.15$, $1.13]$",-0.03,> .999


In [80]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
01mkxf, 04mkkm, 19jgxf, 24jkxf, 26mwxf, 33kkxm, 39mrxm
Removing those cases from the analysis.”


ERROR: Error: No observations remain after removing missing values.
  Try adding to ANOVA call: na.rm = TRUE


In [81]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 04mkkm, 05jwxf, 07zrxf, 10mczf, 12jnxf, 14ktxf, 15agxf, 17makf, 22foxm, 31ltxf, 32mjxf, 36rwxm, 40asxm, 42aszm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“Singular error SSP matrix:
non-sphericity test and corrections not available”


# Condition 64 of 72

`tmin0==0.11 & chan0=="RF" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,5.68,2,2,0.13,.150,.757


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,12,-1.845465,0.9907064,0.2859923
noun_M,10,-2.135666,1.1629144,0.3677458
verb_X,6,-2.842978,0.9977967,0.4073488


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-3.16,"$[-4.07$, $-2.24]$",-11.38,.002
noun_M,noun_M,-1.98,"$[-2.90$, $-1.07]$",-7.15,.007
verb_X,verb_X,-2.79,"$[-3.71$, $-1.88]$",-10.06,.003


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-1.17,"$[-3.27$, $0.92]$",-3.29,.144
noun_F_verb_X,noun_F - verb_X,-0.37,"$[-2.46$, $1.73]$",-1.03,.633
noun_M_verb_X,noun_M - verb_X,0.81,"$[-1.29$, $2.91]$",2.27,.262


In [82]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 08pnxm, 11kbxm, 12jnxf, 13jsxm, 15agxf, 22foxm, 23zdxf, 28pmsf, 31ltxf, 36rwxm, 37lpxm
Removing those cases from the analysis.”


ERROR: Error: No observations remain after removing missing values.
  Try adding to ANOVA call: na.rm = TRUE


In [83]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
03mjkf, 06pppm, 09msxf, 11kbxm, 12jnxf, 14ktxf, 16olxm, 17makf, 18skxm, 19jgxf, 21koxf, 22foxm, 24jkxf, 25abxf, 26mwxf, 27zgxf, 32mjxf, 35mwxm, 36rwxm, 37lpxm, 39mrxm, 40asxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 66 of 72

`tmin0==0.11 & chan0=="LC" & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.01,1.52,3.03,0.05,.434,.008


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,11,-0.6055175,0.6479714,0.1953707
noun_M,14,-0.7874379,0.8798603,0.2351525
verb_X,11,-0.8345657,1.1182691,0.3371708


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,-0.36,"$[-3.34$, $2.62]$",-0.50,.664
noun_M,noun_M,-0.58,"$[-3.56$, $2.40]$",-0.81,.501
verb_X,verb_X,-0.49,"$[-3.47$, $2.49]$",-0.69,.560


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.22,"$[-0.33$, $0.77]$",1.41,.418
noun_F_verb_X,noun_F - verb_X,0.13,"$[-0.42$, $0.69]$",0.86,.692
noun_M_verb_X,noun_M - verb_X,-0.09,"$[-0.64$, $0.47]$",-0.55,.850


In [84]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
01mkxf, 09msxf, 10mczf, 13jsxm, 16olxm, 18skxm, 19jgxf, 24jkxf, 27zgxf, 31ltxf, 35mwxm, 36rwxm, 39mrxm, 42aszm
Removing those cases from the analysis.”


# Condition 67 of 72

`tmin0==0.11 & chan0=="RC" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.91,1.03,2.07,0.30,.228,.296


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,10,0.4077196,1.0046279,0.3176912
noun_M,6,0.4383543,1.1266464,0.4599515
verb_X,8,0.3924365,0.4787236,0.1692544


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,0.13,"$[-0.86$, $1.12]$",0.37,.731
noun_M,noun_M,0.84,"$[-0.15$, $1.83]$",2.45,.076
verb_X,verb_X,0.74,"$[-0.25$, $1.73]$",2.15,.105


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,-0.71,"$[-1.85$, $0.43]$",-2.23,.179
noun_F_verb_X,noun_F - verb_X,-0.61,"$[-1.75$, $0.53]$",-1.90,.251
noun_M_verb_X,noun_M - verb_X,0.10,"$[-1.03$, $1.24]$",0.33,.943


In [85]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
04mkkm, 05jwxf, 08pnxm, 10mczf, 11kbxm, 12jnxf, 14ktxf, 15agxf, 16olxm, 19jgxf, 20jpxf, 22foxm, 25abxf, 30mszf, 31ltxf, 32mjxf, 36rwxm
Removing those cases from the analysis.”


ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


In [86]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
02ppxm, 04mkkm, 05jwxf, 10mczf, 12jnxf, 13jsxm, 17makf, 22foxm, 32mjxf, 36rwxm, 37lpxm, 41roxm
Removing those cases from the analysis.”
Warning message in summary.Anova.mlm(x$Anova):
“Singular error SSP matrix:
non-sphericity test and corrections not available”


# Condition 69 of 72

`tmin0==0.11 & chan0=="LP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.10,2,2,0.93,.907,.079


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,7,2.522520,1.4036655,0.5305357
noun_M,7,3.628220,1.6806887,0.6352406
verb_X,6,2.940462,0.7904734,0.3227094


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,4.16,"$[2.08$, $6.24]$",6.84,.009
noun_M,noun_M,4.03,"$[1.95$, $6.12]$",6.63,.010
verb_X,verb_X,3.73,"$[1.65$, $5.82]$",6.14,.012


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.12,"$[-5.54$, $5.79]$",0.13,.991
noun_F_verb_X,noun_F - verb_X,0.42,"$[-5.24$, $6.09]$",0.44,.903
noun_M_verb_X,noun_M - verb_X,0.30,"$[-5.37$, $5.97]$",0.31,.949


In [87]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
24jkxf, 33kkxm, 34gsxm
Removing those cases from the analysis.”


ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


In [88]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
07zrxf, 09msxf, 13jsxm, 14ktxf, 19jgxf, 24jkxf, 36rwxm, 37lpxm, 42aszm
Removing those cases from the analysis.”


# Condition 71 of 72

`tmin0==0.11 & chan0=="RP" & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.65,1.08,5.42,1.47,.467,.007


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,8,5.618231,3.417587,1.2082994
noun_M,11,5.052340,1.928684,0.5815201
verb_X,9,4.384408,2.825685,0.9418950


### Expected marginal means

,cond0,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F,noun_F,5.84,"$[2.53$, $9.15]$",4.41,.005
noun_M,noun_M,5.48,"$[2.17$, $8.78]$",4.14,.007
verb_X,verb_X,5.26,"$[1.95$, $8.57]$",3.97,.009


### Paired comparisons for expected marginal means

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_noun_M,noun_F - noun_M,0.36,"$[-1.05$, $1.78]$",0.70,.766
noun_F_verb_X,noun_F - verb_X,0.58,"$[-0.83$, $1.99]$",1.13,.520
noun_M_verb_X,noun_M - verb_X,0.22,"$[-1.19$, $1.63]$",0.42,.907


In [89]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message:
“Missing values for following ID(s):
15agxf, 19jgxf, 31ltxf
Removing those cases from the analysis.”


ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


In [90]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

ERROR: Error in dim(ordered) <- ns: dims [product 1] do not match the length of object [0]


In [ ]:
IDX = 71